## Функция ранжирования Okapi BM25

Если у вас нет ассесоров, которые могут проставить оценки релевантности документам, вышепроделанный способ ранжирования вам не подходит. В этом случае можно использовать формулу *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)). Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{(k+1)*f(q_i,D)}{f(q_i,D)+k_1(1-b+b\frac{|D|}{avgdl})} $$ 
где $f(q_i,D)$ - частота слова (TF) $q_i$ в документе $D$, $|D|$ - длина документа (количество слов в нём), а *avgdl* — средняя длина документа в коллекции. 
$$$$
$k_1$ и $b$ — свободные коэффициенты, обычно их выбирают как $k_1$=2.0 и $b$=0.75.
$$$$
$\text{IDF}(q_i)$ есть обратная документная частота (IDF) слова $q_i$: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
где $N$ - общее количество документов в коллекции, а  $n(q_i)$ — количество документов, содержащих $q_i$. 

In [1]:
from inv_index import InvIndex

In [2]:
from os import listdir

In [3]:
from IPython.display import HTML, display
from collections import defaultdict

In [4]:
from math import log

k1 = 2.0
b = 0.75

def score_BM25(n, qf, N, dl, avdl):
    K = compute_K(dl, avdl)
    IDF = log((N - n + 0.5) / (n + 0.5))
    frac = ((k1 + 1) * qf) / (K + qf)
    return IDF * frac


def compute_K(dl, avdl):
    return k1 * ((1-b) + b * (float(dl)/float(avdl)))

**Задание по проекту.** 
Для его выполнения вам понадобится собранная коллекция документов и функция, составляющая обратный индекс по словам в коллекции.

Напишите функцию (или несколько отдельных логичный функций), которая по запросу $Q = q_1,..., g_n$ и коллекции $D$ сортирует выдачу подходящих документов. Будем считать документ подходящим, если он содержит хотя бы одно слово из запроса (из которого удалены стоп-слова). В качестве метрики используйте *Okapi BM25*.

Для проверки работы функции на вашем корпусе используйте запрос **каникулы на новый год и рождество**. Выведите ссылки в ipynb на первые десять докуменов в отсортированной выдаче(как во втором семинаре с помощью IPython.display) и их оценку BM25. Напомню, что ссылки на документы хрянятся в самих доках под тэгом @url.

Про что не забыть:
1. Лемматизируем запрос, удаляем стоп-слова => запрос готов
2. Лемматизируем слова в документах => документы готовы к подсчетам статистик по запросу

In [5]:
class Document():
    def __init__(self, lines):
        self.author = lines[0].strip('@au').strip()
        self.title = lines[1].strip(r'@ti').strip()
        self.date = lines[2].strip(r'@da').strip()
        self.topic = lines[3].strip(r'@topic').strip()
        self.url = lines[4].strip(r'@url').strip()
        self.text = lines[5].strip()
        self.tokens = None

In [47]:
class SearchRank():
    def __init__(self, dir_path):
        self.maker = InvIndex('russian')
        self.read_documents(dir_path)
        
    def read_documents(self, dir_path):
        self.documents = []
        self.unique = []
        sum_dl = 0
        cnt = 1
        for filename in listdir(dir_path):
            f = open(''.join([dir_path, '/', filename]), encoding='utf-8')
            lines = f.readlines()
            doc = Document(lines)
            if doc.title not in self.unique:
                self.documents.append(doc)
                self.unique.append(doc.title)
            if self.documents == []:
                print('no documents in directory')
                self.N = 0
        for document in self.documents:
            print(str(cnt)+'/'+str(len(self.documents)))
            cnt += 1
            document.tokens = self.maker.tokenize(document.text)
            sum_dl += len(document.tokens)
        self.inv_idx = self.maker.inv_index([document.tokens for document in self.documents], defaultdict(set))
        self.N = len(self.documents)
        self.avdl = sum_dl/self.N
        print(str(self.N)+' documents collected')
        
    def query(self, query, num_links = 10, another_dir_path=None):
        query_tokens = self.maker.tokenize(query)
        ranking = defaultdict(int)
        if another_dir_path:
            self.read_documents(another_dir_path)
        if not self.documents:
            print ('pls enter dir_path')
        else:
            for document in self.documents:
                has_query_words = False
                dl = len(document.tokens)
                for q_token in query_tokens:
                    if q_token in self.inv_idx:
                        n = len(self.inv_idx[q_token])
                    else:
                        n = 0
                    qf = document.tokens.count(q_token)
                    if qf > 0:
                        has_query_words = True
                    ranking[document] += score_BM25(n, qf, self.N, dl, self.avdl)
                if not has_query_words:
                    ranking.pop(document, None)   
        print(len(ranking))
        if len(ranking) < num_links:
            num_links = len(ranking)
        for doc in sorted(ranking, key=lambda x: ranking[x], reverse=True)[:num_links]:
            display(doc.url, ranking[doc])

In [48]:
# пробный запрос к маленькому корпусу
rank = SearchRank('app/small_coll')

1/28
2/28
3/28
4/28
5/28
6/28
7/28
8/28
9/28
10/28
11/28
12/28
13/28
14/28
15/28
16/28
17/28
18/28
19/28
20/28
21/28
22/28
23/28
24/28
25/28
26/28
27/28
28/28
28 documents collected


In [49]:
rank.query('каникулы на новый год и рождество')

14


'http://zavety-i.ru/?module=articles&action=view&id=1746'

3.492820404442619

'http://zavety-i.ru/?module=articles&action=view&id=1769'

2.89532209979063

...

0.7386437012773451

'http://zavety-i.ru/?module=articles&action=view&id=1780'

0.7261583263085762

'http://mgazeta.com/category/tak-i-zhivem/kak-ne-popast-pod-razdachu-energetikov/'

0.43106298782661046

In [50]:
rank_big = SearchRank('app/articles_collection')

1/325
2/325
3/325
4/325
5/325
6/325
7/325
8/325
9/325
10/325
11/325
12/325
13/325
14/325
15/325
16/325
17/325
18/325
19/325
20/325
21/325
22/325
23/325
24/325
25/325
26/325
27/325
28/325
29/325
30/325
31/325
32/325
33/325
34/325
35/325
36/325
37/325
38/325
39/325
40/325
41/325
42/325
43/325
44/325
45/325
46/325
47/325
48/325
49/325
50/325
51/325
52/325
53/325
54/325
55/325
56/325
57/325
58/325
59/325
60/325
61/325
62/325
63/325
64/325
65/325
66/325
67/325
68/325
69/325
70/325
71/325
72/325
73/325
74/325
75/325
76/325
77/325
78/325
79/325
80/325
81/325
82/325
83/325
84/325
85/325
86/325
87/325
88/325
89/325
90/325
91/325
92/325
93/325
94/325
95/325
96/325
97/325
98/325
99/325
100/325
101/325
102/325
103/325
104/325
105/325
106/325
107/325
108/325
109/325
110/325
111/325
112/325
113/325
114/325
115/325
116/325
117/325
118/325
119/325
120/325
121/325
122/325
123/325
124/325
125/325
126/325
127/325
128/325
129/325
130/325
131/325
132/325
133/325
134/325
135/325
136/325
137/325
138/325
139/

In [51]:
rank_big.query('каникулы на новый год и рождество')

179


'http://zavety-i.ru/?module=articles&action=view&id=1746'

8.023489647066308

'http://mgazeta.com/category/tak-i-zhivem/kakoy-budet-nyneshnyaya-zima-gadaem-na-mokhnatykh-gusenitsakh-i-kommunalshchikakh/'

4.871213038392074

...

2.257531979730941

'http://mgazeta.com/category/tak-i-zhivem/feyerverk-opasnaya-zabava/'

2.248222315357751

'http://mgazeta.com/category/tak-i-zhivem/volk-zabivaka-za-500-reaktsiya-seti-na-talisman-chm-2018-po-futbolu/'

2.205751771307259